# **Distribución de propiedades por segmento**

In [2]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

## 2022

In [3]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "2022_pachuca" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_nov_2022_pachuca', 'df_alfa_q_oct_2022_pachuca']

In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria','precio']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [5]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

In [6]:
for df in dataframes_list:    
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

In [7]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S,779
1,D1,351
2,C3,302
3,B2,279
4,B1,275
5,B3,250
6,D2,232
7,C2,213
8,D3,189
9,A3,161


In [8]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,779,S,1
1,351,D,1
2,302,C,3
3,279,B,2
4,275,B,1
5,250,B,3
6,232,D,2
7,213,C,2
8,189,D,3
9,161,A,3


In [9]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  3609


,segmento,total_por_segmento,porcentaje_del_total
0,A,438,12.136326
1,B,804,22.277639
2,C,659,18.259906
3,D,772,21.390967
4,E,157,4.350236
5,S,779,21.584927


In [10]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]
# Reordenar la tabla de porcentajes según el orden de segmentos
#resultados = resultados.reindex(orden_segmentos)

segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2022', carpeta='assets/graficas')

fig.show()

In [11]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Asegurar que la columna de segmentos tenga el orden deseado
resultados['segmento'] = pd.Categorical(resultados['segmento'], categories=orden_segmentos, ordered=True)

# Reordenar el DataFrame según el orden de los segmentos
resultados = resultados.sort_values('segmento', ascending=False)
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in segmentos.values else '#A9A9A9' 
                     for i, segmento in enumerate(orden_segmentos)}

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes_totales[segmentos == segmento].sum() if segmento in segmentos.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2022', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()

## 2023

In [12]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "2023_pachuca" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [13]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [14]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S,1478
1,B1,551
2,B2,476
3,B3,420
4,C3,445
5,C2,400
6,D1,566
7,D3,249
8,A1,290
9,A2,318


In [15]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,1478,S,1
1,551,B,1
2,476,B,2
3,420,B,3
4,445,C,3
5,400,C,2
6,566,D,1
7,249,D,3
8,290,A,1
9,318,A,2


In [16]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  6299


,segmento,total_por_segmento,porcentaje_del_total
0,A,926,14.700746
1,B,1447,22.971900
2,C,1126,17.875853
3,D,1145,18.177488
4,E,177,2.809970
5,S,1478,23.464042


In [17]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2023', carpeta='assets/graficas')

fig.show()

In [18]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Asegurar que la columna de segmentos tenga el orden deseado
resultados['segmento'] = pd.Categorical(resultados['segmento'], categories=orden_segmentos, ordered=True)

# Reordenar el DataFrame según el orden de los segmentos
resultados = resultados.sort_values('segmento', ascending=False)
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in segmentos.values else '#A9A9A9' 
                     for i, segmento in enumerate(orden_segmentos)}

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes_totales[segmentos == segmento].sum() if segmento in segmentos.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2023', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()

## 2024

In [19]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "2024_pachuca" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca']

In [20]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Categoria':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [21]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [22]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S1,1265
1,,1173
2,E2,910
3,D2,930
4,D3,710
5,C1,694
6,D1,605
7,C3,587
8,B2,546
9,C2,480


In [23]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,1265,S,1
1,1173,NaN,1
2,910,E,2
3,930,D,2
4,710,D,3
5,694,C,1
6,605,D,1
7,587,C,3
8,546,B,2
9,480,C,2


In [24]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  10477


,segmento,total_por_segmento,porcentaje_del_total
0,A,1070,10.212847
1,B,1454,13.878019
2,C,1761,16.808247
3,D,2245,21.427890
4,E,1714,16.359645
5,L,222,2.118927
6,S,2011,19.194426


In [25]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2024', carpeta='assets/graficas')

fig.show()

In [26]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Asegurar que la columna de segmentos tenga el orden deseado
resultados['segmento'] = pd.Categorical(resultados['segmento'], categories=orden_segmentos, ordered=True)

# Reordenar el DataFrame según el orden de los segmentos
resultados = resultados.sort_values('segmento', ascending=False)
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in segmentos.values else '#A9A9A9' 
                     for i, segmento in enumerate(orden_segmentos)}

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes_totales[segmentos == segmento].sum() if segmento in segmentos.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2024', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()